# sahi + train

In [16]:
import os
from glob import glob
import cv2
import json
import ultralytics
from tqdm import tqdm
import shutil
from sklearn.model_selection import train_test_split

In [54]:
with open('BIRD_v200_3_1/BIRD_v210_3.txt', 'r') as f:
    annot = f.read().split('\n')

In [ ]:
5616 3744

In [56]:
i = 0
bird_annots = []

while i < len(annot):

    if 'jpg' in annot[i]:
        filename = annot[i]
        i += 1
        label = ''
    
    while i < len(annot) and 'jpg' not in annot[i]:
        if annot[i].split(',')[-1] == 'b':
            
            x, y, w, h = [float(x) for x in annot[i].split(',')[:-1]]
            
            x /= 5616
            y /= 3744
            w /= 5616
            h /= 3744
            
            label += '0 ' + ' '.join([str(x), str(y), str(w), str(h)])
            label += '\n'
            i += 1
        else:
            i += 1
    if len(label) > 0:
        bird_annots.append((filename, label))

In [57]:
train_data, val_data = train_test_split(bird_annots, test_size=0.15)

In [59]:
if os.path.exists('bird_data'):
    shutil.rmtree('bird_data/')

In [60]:
os.mkdir('bird_data')
os.mkdir('bird_data/train')
os.mkdir('bird_data/train/images')
os.mkdir('bird_data/train/labels')
os.mkdir('bird_data/val')
os.mkdir('bird_data/val/images')
os.mkdir('bird_data/val/labels')

In [61]:
for x in tqdm(train_data):
    try:
        img = cv2.imread('BIRD_v200_3_1/BIRD_v200_3_1/'+x[0])
        with open('bird_data/train/labels/'+x[0].split('.')[0]+'.txt', 'w') as f:
            f.write(x[1])
        cv2.imwrite('bird_data/train/images/'+x[0], cv2.resize(img, (img.shape[1] // 2, img.shape[0] // 2)))
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████| 597/597 [01:46<00:00,  5.58it/s]


In [62]:
for x in tqdm(val_data):
    try:
        img = cv2.imread('BIRD_v200_3_1/BIRD_v200_3_1/'+x[0])
        with open('bird_data/val/labels/'+x[0].split('.')[0]+'.txt', 'w') as f:
            f.write(x[1])
        cv2.imwrite('bird_data/val/images/'+x[0], cv2.resize(img, (img.shape[1] // 2, img.shape[0] // 2)))
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [00:18<00:00,  5.82it/s]


In [63]:
data_yml = """
names:
- bird
nc: 1
train: D:/code/champs/lct/bird_data/train
val: D:/code/champs/lct/bird_data/val
"""

In [64]:
with open('bird_data/data.yml', 'w') as f:
    f.write(data_yml)

In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s.pt')

# Use the model
model.train(data="bird_data/data.yml", imgsz=2048, batch=8, epochs=100) 

New https://pypi.org/project/ultralytics/8.0.109 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.105  Python-3.8.15 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
yolo\engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=bird_data/data.yml, epochs=100, patience=50, batch=8, imgsz=2048, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, 

AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed 
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias
train: Scanning D:\code\champs\lct\bird_data\train\labels.cache... 352 images, 0 backgrounds, 2 corrupt: 100%|█████████
train: WARNING  D:\code\champs\lct\bird_data\train\images\IMG_2012-09-28T09-08-15.jpg: ignoring corrupt image/label: negative label values [ -0.0035613]
train: WARNING  D:\code\champs\lct\bird_data\train\images\IMG_2012-09-28T09-14-07.jpg: ignoring corrupt image/label: negative label values [ -0.0023148]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning D:\code\champs\lct\bird_data\val\labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58
Plotting labels to runs\detect\train7\labels.jpg... 
Image sizes 2048 train, 2048 val
Using 8 dataloade

FileNotFoundError: Dataset 'bird_data/data.yml' for task=detect not found 

# inference

In [24]:
# !git clone https://github.com/sweetlhare/sahi_yolov8_segment.git

In [33]:
import torch
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.predict import get_sliced_prediction

In [34]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path='runs/detect/train4/weights/best.pt',
    confidence_threshold=0.2,
    device='cuda:0' if torch.cuda.is_available() else 'cpu'
)

In [35]:
from PIL import Image
import numpy as np

In [36]:
detection_model.perform_inference(np.array(Image.open('data/303/p1.jpg')))

In [37]:
for image_ind, image_predictions in enumerate(detection_model._original_predictions):
    break

In [41]:
result = get_sliced_prediction(
    'data/303/p1.jpg',
    detection_model,
    slice_height=4096,
    slice_width=4096,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2
)

Performing prediction on 32 number of slices.


In [31]:
result.object_prediction_list

[]

In [32]:
result.export_visuals(export_dir='images/', file_name='303+p1.jpg')

In [94]:
8192 / 12418

0.6596875503301659

In [ ]:
0.66*img.shape[0]

In [95]:
img = read_image('data/303/p1.jpg')

In [98]:
img.shape[0]

12427

In [118]:
import requests

url = "http://127.0.0.1:8000/process_images/"
resp = {
    'object_id': 10, 
    'paths': ['images/303+p1.jpg.png']
}
response = requests.post(url, json=resp)

print(response.status_code)
print(response.json())

422
{'detail': [{'loc': ['query', 'object_id'], 'msg': 'field required', 'type': 'value_error.missing'}]}
